In [1]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [2]:
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.19.2
    Uninstalling datasets-2.19.2:
      Successfully uninstalled datasets-2.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 re

In [3]:
!pip install zipfile

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [4]:
!pip install -U -q PyDrive

In [5]:
!pip install gdown

In [6]:
file_id = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [7]:
# !gdown --id <File ID>
!gdown --id $file_id

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1KzPx96hawpKg-Iw2yNPW9VnzqbY0uKLh
From (redirected): https://drive.google.com/uc?id=1KzPx96hawpKg-Iw2yNPW9VnzqbY0uKLh&confirm=t&uuid=97001d9d-fbf9-41cf-883a-aff98c4de9ca
To: /kaggle/working/T5_finetuned_SummaryFlow_Epoch_29.zip
100%|████████████████████████████████████████| 994M/994M [01:23<00:00, 11.8MB/s]


In [8]:
trianEpochs = 2

In [9]:
import os
def get_most_recent_file(directory='.'):
    files = os.listdir(directory)
    paths = [os.path.join(directory, basename) for basename in files]
    return max(paths, key=os.path.getctime)

# Find the most recently downloaded file
downloaded_file = get_most_recent_file()

print(f"Downloaded file: {downloaded_file}")

Downloaded file: ./__notebook__.ipynb


In [10]:
downloaded_file = '/kaggle/working/T5_finetuned_SummaryFlow_Epoch_29.zip'

In [11]:
import pandas as pd
import wandb
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset

2024-06-28 02:45:23.475149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 02:45:23.475288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 02:45:23.599386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
os.environ['WANDB_SILENT'] = 'true'

In [13]:
# Device Selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [15]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [16]:
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [17]:
import os
import zipfile

# Path to the ZIP file containing the model
model_zip_path = downloaded_file

if downloaded_file.endswith('.zip'):
    # Create the directory to extract the model
    os.makedirs('/kaggle/working/model', exist_ok=True)

# Extract the model from the ZIP file
    with zipfile.ZipFile(model_zip_path, 'r') as zip_ref:
        zip_ref.extractall('/kaggle/working/model')
else:
    raise ValueError(f"The downloaded file is not a zip file: {downloaded_file}")

In [18]:
# Define the path to the epoch file
epoch_file_path = '/kaggle/working/model/epoch.txt'

# Read the epoch number from the file
with open(epoch_file_path, 'r') as f:
    epoch_number = f.read().strip()

# Convert the read value to an integer
epoch_number = int(epoch_number)

epoch_number = epoch_number + trianEpochs
print(f"The Current epoch number is: {epoch_number}")



The Current epoch number is: 31


In [19]:
os.remove(epoch_file_path)
print(f"The file {epoch_file_path} has been deleted.")

The file /kaggle/working/model/epoch.txt has been deleted.


In [20]:
torch.cuda.empty_cache()

In [21]:
model_checkpoint = '/kaggle/working/model'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [22]:
# Read Data From Excel Files
train_df = pd.read_excel(r'/kaggle/input/new-summary-flow-data/TrainData.xlsx')
val_df = pd.read_excel(r'/kaggle/input/new-summary-flow-data/ValidationData.xlsx')

In [23]:
# Data Preprocessing
def preprocess_text(text):
    return text.strip("[]").replace("'", "").replace("\"", "")

train_df['summary'] = train_df['summary'].apply(preprocess_text)
val_df['summary'] = val_df['summary'].apply(preprocess_text)

In [24]:
# Put The Data In Form Of Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [25]:
# Data Preprocessing
def get_feature(batch):
    encodings = tokenizer(batch['text'], text_target=batch['summary'], max_length=1024, truncation=True)
    encodings = {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': encodings['labels']}
    return encodings

train_dataset = train_dataset.map(get_feature, batched=True)
val_dataset = val_dataset.map(get_feature, batched=True)

Map:   0%|          | 0/110374 [00:00<?, ? examples/s]

Map:   0%|          | 0/14411 [00:00<?, ? examples/s]

In [26]:
# Put The Data In Form Of Torch Tensors
columns = ['input_ids', 'labels', 'attention_mask']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)

In [27]:
# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [28]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/T5-model',  # Output directory for checkpoints and evaluation results
    overwrite_output_dir=False,  # Do not overwrite the output directory
    num_train_epochs=trianEpochs,  # Number of training epochs to run
    per_device_train_batch_size=1,  # Batch size per GPU
    per_device_eval_batch_size=1,  # Batch size for evaluation per GPU
    warmup_steps=500,  # Number of warmup steps
    weight_decay=0.01,  # Weight decay for regularization
    logging_steps=10,  # Log every N steps
    evaluation_strategy='steps',  # Evaluate every `eval_steps` steps
    eval_steps=3000,
    save_steps=6000,# Number of steps between evaluations  # Save checkpoint every N steps
    gradient_accumulation_steps=8,  # Number of gradient accumulation steps
    load_best_model_at_end=True,  # Load the best model from the checkpoint at the end of training
    metric_for_best_model="eval_loss",  # Metric to use for determining the best model
    greater_is_better=False,
    resume_from_checkpoint='latest_checkpoint'
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
# Use GPU
model = model.to(device)

# Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [30]:
wandb.login(key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

True

In [31]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
3000,0.399600,7.512329
6000,0.508600,7.078584
9000,0.660400,6.195838
12000,0.915900,5.344384
15000,1.064500,4.951590
18000,1.204000,4.397228
21000,1.650700,3.874937
24000,2.559600,3.217551
27000,2.727700,2.879269


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=27592, training_loss=1.215158411811213, metrics={'train_runtime': 36365.8312, 'train_samples_per_second': 6.07, 'train_steps_per_second': 0.759, 'total_flos': 6.666468043957862e+16, 'train_loss': 1.215158411811213, 'epoch': 1.9998912787431822})

In [32]:
model_name = f'T5_finetuned_SummaryFlow_Epoch_{epoch_number}'

In [33]:
# Save the trained model
model_save_path = f'/kaggle/working/{model_name}'
trainer.save_model(model_save_path)

In [34]:
epoch_file_path = f'{model_save_path}/epoch.txt'

with open(epoch_file_path, 'w') as f:
    f.write(str(epoch_number))

In [35]:
import zipfile
import os
# Define the folder to zip
folder_to_zip = f'/kaggle/working/{model_name}'

# Define the path for the zip file
zip_file_path = f'/kaggle/working/{model_name}.zip'

# Create a Zip file
with zipfile.ZipFile(zip_file_path, 'w') as zip_file:
    for folder_name, subfolders, file_names in os.walk(folder_to_zip):
        for file_name in file_names:
            file_path = os.path.join(folder_name, file_name)
            zip_file.write(file_path, os.path.relpath(file_path, folder_to_zip))


In [36]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Load the service account credentials
creds = Credentials.from_service_account_file('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

# Build the drive service
drive_service = build('drive', 'v3', credentials=creds)

# Create a media file upload object
media = MediaFileUpload(zip_file_path, mimetype='application/zip')

# Create a new file on Google Drive
file_metadata = {
    'name': f'{model_name}.zip',
    'mimeType': 'application/zip'
}
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

print('Uploaded file with ID {}'.format(file.get('id')))

Uploaded file with ID 1bZOqEoBG5z_0cS46-Z5HtpGJr3MFeR-2


In [37]:
from googleapiclient.errors import HttpError

# After uploading the file...
try:
    def callback(request_id, response, exception):
        if exception:
            # Handle error
            print(exception)
        else:
            print("Permission Id: %s" % response.get('id'))

    batch = drive_service.new_batch_http_request(callback=callback)
    anyone_permission = {
        'type': 'anyone',
        'role': 'reader',
    }
    batch.add(drive_service.permissions().create(
        fileId=file.get('id'),
        body=anyone_permission,
        fields='id',
    ))
    batch.execute()
except HttpError as error:
    print(f'An error occurred: {error}')

Permission Id: anyoneWithLink


In [38]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Load the service account credentials
creds = Credentials.from_service_account_file('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')

# Build the drive service
drive_service = build('drive', 'v3', credentials=creds)

# List the first 100 files in the service account's Google Drive
results = drive_service.files().list(pageSize=100, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(f'{item["name"]} ({item["id"]})')

Files:
T5_finetuned_SummaryFlow_Epoch_31.zip (1bZOqEoBG5z_0cS46-Z5HtpGJr3MFeR-2)
T5_finetuned_SummaryFlow_Epoch_29.zip (1KzPx96hawpKg-Iw2yNPW9VnzqbY0uKLh)
T5_finetuned_SummaryFlow_Epoch_27.zip (1nQpLvo5rL-HDXjMzQPhBL5Mk6dAfNYH_)
T5_finetuned_SummaryFlow_Epoch_25.zip (122q8Z5hhMo_VIO2vdDXDsYzx-jO4481V)
T5_finetuned_SummaryFlow_Epoch_23.zip (1u8IxiUbKYg1ZANw9KqT5d7CONYQyge0g)
T5_finetuned_SummaryFlow_Epoch_17.zip (1yJKvsh_kNN1KQRix7LW3j0k0YWpw74IZ)
T5_finetuned_SummaryFlow_Epoch_21.zip (1tGBukZr9cJcghD-97wEiyxp31FaJwGW1)
T5_finetuned_SummaryFlow_Epoch_19.zip (1pnsW3WnJTSqdG9SlAkKOxmjcmsGhSjqD)
T5_finetuned_SummaryFlow_Epoch_13.zip (1qM2-261qCn5FZ8PboeBBocjJJXABBsnA)
T5_finetuned_SummaryFlow_Epoch_8.zip (1mxOoSbjzlBqeA0LseA01z_7GqZQrn3d4)
T5_finetuned_SummaryFlow_Epoch_2.zip (1LueoxBELeigSrNCKKsIU8GgfGXqW6HaU)
T5_finetuned_SummaryFlow_Epoch_1.zip (1GG66zYU4u-uFTRjxIRURkYBnDJdb7obA)
SummaryFlow_Run_8.zip (1mnp_3qv41lbG6VKanGI4Woj1S6FdehuS)
SummaryFlow_Run.zip (1Lt7nNHffIEldUoFlNdWqqi1cc5K7